### 库

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from tqdm.notebook import tqdm, trange, tnrange
import json, re

In [2]:
import numpy as np
import pandas as pd
import os, sys
from time import time, sleep

In [3]:
from vVariableInspector import _getshapeof
from vUtil.vFile import fprint, readlines, linesReader
from vUtil.vEmail import sendEmail
from vUtil.vTime import convertSeconds, getNow

In [4]:
from vMysql import MysqlProxy

In [5]:
from util import frmt, rmUnseen, groupConcat, deleteIndexRows
from cfg import host, master, port, contType
from cfg import dbHost, dbUser, dbPwd
from cfg import ncols

### mysql、es

In [6]:
index = 'landinn_product'

In [7]:
es=Elasticsearch(hosts=host,port=port)

In [8]:
db = MysqlProxy(ip=dbHost, user=dbUser, password=dbPwd)

### 函数

In [9]:
sTmpProduct = '''
create temporary table tmp_product
(
    select golaxy_product_id as gid, product_name as name, hangye, yewu, setup_time, 
        cast(affiliation_id as char) as enterprises
    from product 
    where !ifnull(is_deleted,0)
    limit %d, %d
);
'''
sRmTmpProduct = '''
drop temporary table tmp_product;
'''

In [10]:
sSelectData = '''
select  gid, name, hangye, yewu, setup_time, enterprises
from tmp_product;
'''

In [11]:
def getActions (data):
    actions = []    
    for i in range(len(data)):
        action={'_op_type':'index',#操作 index update create delete  
            '_index':index,#index
            '_id' : data['gid'][i],
            '_source':
           {
                "id" : data['gid'][i],
                "name" : rmUnseen(data['name'][i], None),
                "yewu" : rmUnseen(data['yewu'][i], None),
                "hangye" : rmUnseen(data['hangye'][i], None),
                "enterprises" : data['enterprises'][i],
                "setup_time" : None if data['setup_time'][i] is pd.NaT else data['setup_time'][i],
            }
        }
        actions.append(action)
    return actions

### 索引

In [12]:
table = 'product'

In [13]:
deleteIndexRows(db, es, table, 'golaxy_product_id', index=index)

{'landinn_product': 0}

In [24]:
sizeBulk = 100
now = 0
nProduct = db.count(table, where='!ifnull(is_deleted,0)').values.item()

In [13]:
# now = 120

In [25]:
now, sizeBulk, nProduct

(0, 100, 972)

In [18]:
startOfAll = time()
for i in trange(now, nProduct, sizeBulk, ncols=ncols):
    now = i
    
    db.sql(sTmpProduct % (now, sizeBulk))
    
    data = db.sql(sSelectData)
    db.sql(sRmTmpProduct)
    db.close()
    
    actions = getActions(data)
    if actions: helpers.bulk(client=es,actions=actions)
sendEmail(f'insert cost time {convertSeconds(time() - startOfAll)}', 'insert landinn es products complete')

### 关闭

In [21]:
es.close()

In [22]:
db.close()